In [1]:
import random
import numpy as np
import pandas as pd
import manage_data as md
import managePeakData as mpd
import preprocessing as pp
%matplotlib inline
import matplotlib.pyplot as plt

In [62]:
learning_coils = [0,1,28,44,8,9,7,49,50,78,79,81]
testing_coils = list(set(range(88)) - set(learning_coils) - set({31}))

In [63]:
def concat_df(coil_list):
    big_df = mpd.importPeak(coil_list[0]).dropna()
    for coil in coil_list[1:]:
        df = mpd.importPeak(coil).dropna()
        big_df = pd.concat([big_df, df], ignore_index=True)
    return big_df

In [64]:
df = concat_df(learning_coils)
sticking_index = list(df[df.sticking==True].index)
non_sticking_index = list(df[df.sticking==False].index)
group_of_items = non_sticking_index               # a sequence or set will work here.
num_to_select = len(df[df.sticking==True])        # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)
list_of_random_items
ind = sticking_index+list_of_random_items
df_random = df.loc[ind]
print (len(df_random[df_random.sticking==True]))
df_random.groupby('coil').size()

155


coil
0     25
1     21
7     28
8     24
9     20
28    28
44    27
49    29
50    25
78    29
79    26
81    28
dtype: int64

In [65]:
len(df[df.sticking==True])

155

In [66]:
df_random.head()

,Ximf0,Ximf1,Ximf2,Yimf0,Yimf1,Yimf2,coil,speed,sticking,thickness,time_begin,time_end
63,1.60016,4.00040,4.00040,2.159633,15.810453,15.122418,28,1198.821656,True,0.7,96.0,100.0
64,4.00040,4.00040,4.00040,9.725138,39.541426,78.219214,28,1198.593447,True,0.7,100.0,104.0
65,4.00040,4.00040,4.00040,28.248364,40.767361,56.944011,28,1198.680533,True,0.7,104.0,108.0
66,4.00040,4.00040,4.00040,33.609731,58.345937,58.870689,28,1198.696552,True,0.7,108.0,112.0
67,4.40044,4.40044,4.40044,68.611115,70.142810,57.679230,28,1198.534788,True,0.7,112.0,116.0


In [67]:
def create_DB(df):
    feature_col = ['Ximf0', 'Ximf1', 'Ximf2', 'Yimf0', 'Yimf1', 'Yimf2', 'thickness', 'speed']
    X = df[feature_col].values
    Y = df[['sticking']]
    Y = np.array(Y.sticking.values[:], dtype=bool)
    return X, Y

In [68]:
X_train, Y_train = create_DB(df_random)
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [69]:
df = concat_df(testing_coils)
sticking_index = list(df[df.sticking==True].index)
non_sticking_index = list(df[df.sticking==False].index)
group_of_items = non_sticking_index               # a sequence or set will work here.
num_to_select = len(df[df.sticking==True])        # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)
list_of_random_items
ind = sticking_index+list_of_random_items
df_random_test = df.loc[ind]
print (len(df_random[df_random.sticking==False])==len(df_random[df_random.sticking==True]))
df_random_test.groupby('sticking').size()

True


sticking
False    228
True     228
dtype: int64

In [78]:
X_train, Y_train = create_DB(df_random)
X_test, Y_true = create_DB(df_random_test)
from sklearn import tree
from sklearn import metrics
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)
Y_predict = clf.predict(X_test)
m = metrics.confusion_matrix(Y_true, Y_predict, labels=[True, False])
m

array([[156,  72],
       [136,  92]])

In [71]:
df[df.sticking==True].groupby('coil').size()

coil
2      8
3     14
4      6
13     6
20    11
21     7
29    13
34     4
37     5
38    14
40    11
43     2
52     4
53     8
62    10
65    18
66     9
71     8
72    15
74    12
80    15
82    16
83    12
dtype: int64

In [72]:
df = mpd.importPeak(28).dropna()

In [73]:
df.columns.values

array(['Ximf0', 'Ximf1', 'Ximf2', 'Yimf0', 'Yimf1', 'Yimf2', 'coil',
       'speed', 'sticking', 'thickness', 'time_begin', 'time_end'], dtype=object)

In [74]:
def create_XY_df(coil_list):
    feature_col = ['Ximf0', 'Ximf1', 'Ximf2', 'Yimf0', 'Yimf1', 'Yimf2', 'thickness', 'speed']
    X = pd.DataFrame(columns=feature_col)
    Y = pd.DataFrame(columns=['sticking'])
    for coil in coil_list:
        df = mpd.importPeak(coil).dropna()
        X = X.append(df[feature_col])
        Y = Y.append(df[['sticking']])
    X_out = X.values
    Y_out = np.array(Y.sticking.values[:], dtype=bool)
    return X,Y

def create_XY(coil_list):
    X, Y = create_XY_df(coil_list)
    X_out = X.values
    Y_out = np.array(Y.sticking.values[:], dtype=bool)
    return X_out, Y_out

In [75]:
learning_coils = [0,1,28,44,8,9]
X_train, Y_train = create_XY(learn_list)
X_train.shape, Y_train.shape

NameError: name 'learn_list' is not defined

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [ ]:
coil_list = [54,27,10,11,6,7,19]
coil_list = range(2,8)+range(10,28)+range(32,44)+range(45,88)
X_test, Y_true = create_XY(coil_list)
Y_predict = clf.predict(X_test)

In [ ]:
from sklearn import metrics
m = metrics.confusion_matrix(Y_true, Y_predict, labels=[True, False])
m

In [ ]:
coil_list = [54,27,10,11,6,7,19,20,21,33]
testing_coils = range(2,8)+range(10,28)+range(32,44)+range(45,88)
sum_true = []
sum_predict = []
for coil in test_list:
    X_test, Y_true = create_XY([coil])
    Y_predict = clf.predict(X_test)
    sum_true.append(sum(Y_true))
    sum_predict.append(sum(Y_predict))
len(sum_true)

In [ ]:
diff = np.abs(np.array(sum_true)-np.array(sum_predict))
sum(diff<=5)*1./len(diff)*100

In [ ]:
x = sum_true
y = sum_predict
bins = range(len(sum_true))
plt.figure(figsize=(16,9))
plt.bar(bins, x, alpha=0.5, label='Truth')
plt.bar(bins, y, alpha=0.5, label='Prediction', color='g')
plt.legend(loc='upper right')
plt.xlabel('Coil')
plt.ylabel('Number of sticking sequences')
plt.title('Learning coils '+str(learn_list)+' with confusion matrix \n'+str(m))
plt.show()
plt.savefig('./images/bars/learning'+str(learn_list)+'.png')

In [ ]:
is_sticking = np.greater(sum_true,0)
is_predicted = np.greater(sum_predict,1)

In [ ]:
m = metrics.confusion_matrix(is_sticking, is_predicted, labels=[True, False])
m

In [ ]:
x = is_sticking==is_predicted
y = is_predicted
bins = range(len(sum_true))
plt.figure(figsize=(16,9))
plt.bar(bins, x, alpha=0.5, label='Truth')
#plt.bar(bins, y, alpha=0.5, label='Prediction', color='g')
plt.legend(loc='upper right')
plt.xlabel('Coil')
plt.ylabel('Sticking label')
plt.title('Learning coils '+str(learn_list)+' with confusion matrix \n'+str(m))
plt.show()
plt.savefig('./images/bars/prediction '+str(learn_list)+'.png')

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    #tick_marks = np.arange(len(iris.target_names))
    #plt.xticks(tick_marks, iris.target_names, rotation=45)
    #plt.yticks(tick_marks, iris.target_names)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(m)

In [ ]:
df['sticking'==True]

In [ ]:
df.sort_values('sticking')

In [ ]:
type(set())

In [ ]:
import random
group_of_items = {1, 2, 3, 4}               # a sequence or set will work here.
num_to_select = 2                           # set the number to select here.
list_of_random_items = random.sample(group_of_items, num_to_select)
first_random_item = list_of_random_items[0]
second_random_item = list_of_random_items[1] 

In [ ]:
list_of_random_items